In [ ]:
import openpyxl
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import folium
from folium import plugins
from folium.plugins import HeatMap, MarkerCluster
from IPython.display import IFrame
import geopandas as gpd
from shapely import wkt
from shapely.geometry import Point
from math import radians, sin, cos, sqrt, atan2
import branca
import requests

In [ ]:
def get_elevation(lat, lon):
    url = f"https://api.open-elevation.com/api/v1/lookup?locations={lat},{lon}"
    response = requests.get(url).json()
    return response['results'][0]['elevation'] if "results" in response else None

def get_nearest_coastline(lat, lon):
    query = f"""
    [out:json];
    (
      way[natural=coastline](around:50000,{lat},{lon});
    );
    out geom;
    """
    url = "https://overpass-api.de/api/interpreter"
    response = requests.get(url, params={"data": query}).json()
    
    if "elements" in response and response["elements"]:
        coast = response["elements"][0]["geometry"][0]  # First coastline point
        coast_lat, coast_lon = coast["lat"], coast["lon"]
        return Point(coast_lat, coast_lon)
    return None

def haversine_distance(point1: Point, point2: Point) -> float:
    R = 6371.0
    lon1, lat1 = point1.x, point1.y
    lon2, lat2 = point2.x, point2.y
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    delta_lat = lat2 - lat1
    delta_lon = lon2 - lon1
    a = sin(delta_lat / 2)**2 + cos(lat1) * cos(lat2) * sin(delta_lon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c

def create_point(row):
    return Point(row['longitude'], row['latitude'])

In [ ]:
# Load the Excel files into Pandas DataFrames
gis_weather_station = pd.read_csv('./data/gis_weatherstation_shape_2024_10_04.csv')
station_summary_snapshot = pd.read_csv('./data/src_wings_meteorology_station_summary_snapshot_2023_08_02.csv')
windspeed_snapshot = pd.read_csv('./data/src_wings_meteorology_windspeed_snapshot_2023_08_02.csv')

src_vri_snapshot = pd.read_csv('./data/src_vri_snapshot_2024_03_20.csv')
nam = pd.read_csv('./data/nam.csv')

In [ ]:
gis_weather_station['geometry'] = gis_weather_station['shape'].apply(wkt.loads)
gis_weather_station_gpd = gpd.GeoDataFrame(gis_weather_station, geometry='geometry', crs=f"EPSG:{gis_weather_station['shape_srid'][0]}")

src_vri_snapshot['geometry'] = src_vri_snapshot['shape'].apply(wkt.loads)
src_vri_snapshot_gpd = gpd.GeoDataFrame(src_vri_snapshot, geometry='geometry', crs=f"EPSG:{src_vri_snapshot['shape_srid'][0]}")

gis_weather_station_gpd = gis_weather_station_gpd.to_crs(src_vri_snapshot_gpd.crs)

print(f"Weather Station CRS:    {gis_weather_station_gpd.crs}")
print(f"VRI Polygon CRS:        {src_vri_snapshot_gpd.crs}")

In [ ]:
# Get elevation and nearest coastline for Weather Station
gis_weather_station_gpd['elevation_m'] = gis_weather_station_gpd.apply(lambda row: get_elevation(row['geometry'].y, row['geometry'].x), axis=1)
gis_weather_station_gpd['nearest_coastline_geometry'] = gis_weather_station_gpd.apply(lambda row: get_nearest_coastline(row['geometry'].y, row['geometry'].x), axis=1)

# Save file as CSV
gis_weather_station_gpd.to_csv("data/gis_weather_station.csv", index=False)

In [ ]:
# Get ESPG:4326
nam_crs = src_vri_snapshot['shape_srid'][0]

# Create NAM Geodataframe
nam['geometry'] = nam.apply(lambda row: Point(row['longitude'], row['latitude']), axis=1)
nam_gpd = gpd.GeoDataFrame(nam, geometry='geometry', crs=nam_crs)

# Filter NAM within the VRI polygons
nam_vri_gpd = gpd.sjoin(nam_gpd, src_vri_snapshot_gpd, how='inner', predicate='within')

In [ ]:
# Get the unique geometries within the VRI
nam_vri_gpd_geometry = nam_vri_gpd[['geometry']].drop_duplicates().reset_index(drop=True)

# Get elevation and nearest coastline for the NAM points
nam_vri_gpd_geometry['elevation_m'] = nam_vri_gpd_geometry.apply(lambda row: get_elevation(row['geometry'].y, row['geometry'].x), axis=1)
nam_vri_gpd_geometry['nearest_coastline_geometry'] = nam_vri_gpd_geometry.apply(lambda row: get_nearest_coastline(row['geometry'].y, row['geometry'].x), axis=1)

# Save file as CSV
nam_vri_gpd_geometry.to_csv("data/NAM_vri_additional_columns.csv")